In [1]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
from time import time
from pathlib import Path
from collections import defaultdict

# Torch and Torchvision
import torch
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torch.utils.tensorboard import SummaryWriter

# Own functions and logging of git state
repo_path = "/home/ubuntu/source/dermx-experiments/"
sys.path.append(repo_path)
from dermx.utils import make_dir, save_json, create_and_save_class_map, get_git_info
from dermx.models import DermXModelGuidedAttention
from dermx.model_utils import freeze_layers
from dermx.datasets import DermXAttnDataset
from dermx.train_utils import (
    iterate_through_attn_dataloader,
    print_history_result,
    str_optimizer_map,
    str_scheduler_map,
)
from dermx.losses import str_loss_map

## Hyperparameters

In [2]:
diagnoses = [
    "Acne",
    "Actinic keratosis",
    "Psoriasis",
    "Seborrheic dermatitis",
    "Viral warts",
    "Vitiligo",
]
characteristics = [
    "Plaque",
    "Papule",
    "Dermatoglyph disruption",
    "Pustule",
    "Scar",
    "Closed comedo",
    "Open comedo",
    "Patch",
    "Sun damage",
    "Scale",
    # "Macule",
    # "Thrombosed capillaries",
    # "Telangiectasia",
    # "Nodule",
    # "Cyst",
    # "Leukotrichia",
]
diagnoses.sort()
characteristics.sort()

hps = {
    # Model
    "backbone_name": "efficientnet_b2",
    "use_pretrained": True,
    "freeze": False,
    "unfreeze_layers": [],
    "unfreeze_bn_layers": True,
    "num_units_in_dx_clf": 64,
    "num_units_in_cx_clf": 64,
    "dropout": 0.2,
    "use_skip_connection": True,
    "use_downsampled_masks": False,
    "use_batchnorm": False,
    "return_gradcam_attributes": True,
    # Training, loss and optimizer
    "loss": {"dx": "ce", "cx": "bce", "attn": "softsesp"},
    "loss_factors": {"dx": 1.0, "cx": 0.0, "attn": 0.0},
    "loss_kwargs": {"dx": {}, "cx": {}, "attn": {"activation": None}},
    "class_weights": {"dx": None, "cx": None, "attn": None},
    "optimizer": "adamw",
    "optimizer_kwargs": {},
    "learning_rate": 0.0005,
    "lr_scheduler": "cosine_annealing_warm_restarts",
    "lr_scheduler_kwargs": {"T_0": 3, "T_mult": 3, "eta_min": 1e-5},
    "batch_size": 32,
    "dataaug_color_jitter_kwargs": {
        "brightness": 0.35,
        "contrast": 0.2,
        "saturation": 0.2,
        "hue": 0.15,
    },
    "dataaug_random_affine_kwargs": {
        "degrees": 10,
        "scale": (0.85, 1.15),
        "translate": (0.15, 0.15),
    },
    # Data
    # "target_size": (256, 256),
    "diagnoses": diagnoses,
    "characteristics": characteristics,
    "num_segm_classes": len(characteristics),
    "num_dx_classes": len(diagnoses),
    "norm_mean": [0.485, 0.456, 0.406],
    "norm_std": [0.229, 0.224, 0.225],
    # IO
    "experiment_root_dir": "/home/ubuntu/store/MedIA/experiments_rebuttal/class_weights",
    "label_map_dir": "/home/ubuntu/store/dermx-folds",
    "image_dir": "/home/ubuntu/store/dermx_cleaned_images",
    "mask_dir": "/home/ubuntu/store/dermx_cleaned_masks/fusion_masks",
    "mask_label_fusion_rule": "fuzzy_relative",
    "checkpoint_metric": "loss",
    "git": get_git_info(repo_path),
}

# Set parameters according to backbones
if hps["backbone_name"] == "efficientnet_b0":
    hps["gradcam_target_layer"] = "backbone.blocks.6.0.conv_pwl"
    hps["target_size"] = (224, 224)
    hps["mask_downsample_size"] = (7, 7) if hps["use_downsampled_masks"] else None
elif hps["backbone_name"] == "efficientnet_b1":
    hps["gradcam_target_layer"] = "backbone.blocks.6.1.conv_pwl"
    hps["target_size"] = (240, 240)
    hps["mask_downsample_size"] = (8, 8) if hps["use_downsampled_masks"] else None
elif hps["backbone_name"] == "efficientnet_b2":
    hps["gradcam_target_layer"] = "backbone.blocks.6.1.conv_pwl"
    hps["target_size"] = (260, 260)
    hps["mask_downsample_size"] = (9, 9) if hps["use_downsampled_masks"] else None
elif hps["backbone_name"] == "resnet50":
    hps["gradcam_target_layer"] = "backbone.layer4.2.conv3"
    hps["target_size"] = (256, 256)
    hps["mask_downsample_size"] = (9, 9) if hps["use_downsampled_masks"] else None

# Calculate number of epochs
if hps["lr_scheduler"] == "cosine_annealing_warm_restarts":
    num_lr_scheduler_cycles = 4
    hps["num_epochs"] = sum(
        hps["lr_scheduler_kwargs"]["T_0"]
        * hps["lr_scheduler_kwargs"]["T_mult"] ** cycle_idx
        for cycle_idx in range(num_lr_scheduler_cycles)
    )

## Experiment directories

In [9]:
experiment_dir = make_dir(
    hps["experiment_root_dir"], f"{hps['backbone_name']}_{int(time())}"
)
fold_idx = "0"
experiment_fold_dir = make_dir(experiment_dir, f"fold_{fold_idx}")
save_json(experiment_dir, "hyperparameters.json", hps)

## Parameters that can be set outside the training loop

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transforms
rgb_transforms_train = [
    T.ColorJitter(**hps["dataaug_color_jitter_kwargs"]),
    T.Normalize(mean=hps["norm_mean"], std=hps["norm_std"]),
]
shared_transforms_train = [
    T.RandomHorizontalFlip(),
    T.RandomVerticalFlip(),
    T.RandomAffine(**hps["dataaug_random_affine_kwargs"]),
    T.Resize(hps["target_size"]),
]
rgb_transforms_test = [T.Normalize(mean=hps["norm_mean"], std=hps["norm_std"])]
shared_transforms_test = [T.Resize(hps["target_size"])]

# Metrics
metric_names = list(hps["loss"].keys()) + ["loss", "dx_acc", "cx_f1", "soft_iou"]
checkpoint_metric = hps["checkpoint_metric"]
metric_larger_is_better = {
    name: name in ("dx_acc", "cx_f1", "soft_iou") for name in metric_names
}
checkpoint_metric_value = {
    name: 0.0 if metric_larger_is_better[name] else np.inf for name in metric_names
}

## Label and class maps

In [11]:
folds_dir = os.path.join(hps["label_map_dir"], f"fold_{fold_idx}")
train_df = pd.read_csv(
    os.path.join(folds_dir, f"metadata_fold_{fold_idx}_train.csv")
)
test_df = pd.read_csv(
    os.path.join(folds_dir, f"metadata_fold_{fold_idx}_test.csv")
)
class_map_dx = create_and_save_class_map(hps["diagnoses"], "dx", experiment_dir)
class_map_char = create_and_save_class_map(
    hps["characteristics"], "char", experiment_dir
)

In [12]:
def calc_class_weights(class_counts):
    """
    Calculate class weighs dictionary to use as input for the cnn training. This is useful if the training set is
    imbalanced.

    The weight of class "i" is calculated as the number of samples in the most populated class divided by the number of
    samples in class i (max_class_frequency / class_frequency).
    Note that the class weights are capped at 10. This is done in order to avoid placing too much weight on
    small fraction of the dataset. For the same reason, the weight is set to 1 for any class in the training set that
    contains fewer than 5 samples.

    :param class_counts: A list with the number of files for each class.
    :return:
    """

    # Fixed parameters
    class_weights = []
    max_freq = max(class_counts.values)
    class_weights = [max_freq / count for count in class_counts.values]
    
    print("Classes: " + str(len(class_counts.keys())))
    print("Samples per class: " + str(class_counts.values))
    print("Class weights: " + str(class_weights))

    return class_weights


In [13]:
class_weights = calc_class_weights(train_df['diagnosis'].value_counts())

Classes: 6
Samples per class: [108  88  85  81  77  59]
Class weights: [1.0, 1.2272727272727273, 1.2705882352941176, 1.3333333333333333, 1.4025974025974026, 1.8305084745762712]


## Training initialization

### Dataset and Dataloaders

In [14]:
# Train
data_dermx_train = DermXAttnDataset(
    train_df,
    class_map_dx,
    class_map_char,
    image_dir=hps["image_dir"],
    mask_dir=hps["mask_dir"],
    mask_label_fusion_rule=hps["mask_label_fusion_rule"],
    rgb_transforms=rgb_transforms_train,
    shared_transforms=shared_transforms_train,
    mask_downsample_size=hps["mask_downsample_size"],
)
dataloader_train = DataLoader(
    data_dermx_train, batch_size=hps["batch_size"], shuffle=True, num_workers=8
)

# Test
data_dermx_test = DermXAttnDataset(
    test_df,
    class_map_dx,
    class_map_char,
    image_dir=hps["image_dir"],
    mask_dir=hps["mask_dir"],
    mask_label_fusion_rule=hps["mask_label_fusion_rule"],
    rgb_transforms=rgb_transforms_test,
    shared_transforms=shared_transforms_test,
    mask_downsample_size=hps["mask_downsample_size"],
)
dataloader_test = DataLoader(
    data_dermx_test, batch_size=hps["batch_size"], shuffle=True, num_workers=8
)

### Model

In [15]:
model = DermXModelGuidedAttention(
    num_dx_classes=hps["num_dx_classes"],
    num_cx_classes=hps["num_segm_classes"],
    target_layer=hps["gradcam_target_layer"],
    backbone_name=hps["backbone_name"],
    pretrained=hps["use_pretrained"],
    return_gradcam_attributes=hps["return_gradcam_attributes"],
    interpolate_gradcam_attributes=(not hps["use_downsampled_masks"]),
    num_units_in_dx_clf=hps["num_units_in_dx_clf"],
    num_units_in_cx_clf=hps["num_units_in_cx_clf"],
    dropout=hps["dropout"],
    use_batchnorm=hps["use_batchnorm"],
    use_skip_connection=hps["use_skip_connection"],
    device=device,
)
model = model.to(device)
if hps["freeze"]:
    freeze_layers(
        model,
        ignore_names=hps["unfreeze_layers"],
        ignore_bn=hps["unfreeze_bn_layers"],
    )

In [16]:
model

DermXModelGuidedAttention(
  (backbone): EfficientNetFeatures(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=1e-05, 

### Optimizer

In [11]:
Optimizer = str_optimizer_map[hps["optimizer"]]
optimizer = Optimizer(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=hps["learning_rate"],
    **hps["optimizer_kwargs"],
)
if hps["lr_scheduler"] is not None:
    Scheduler = str_scheduler_map[hps["lr_scheduler"]]
    scheduler = Scheduler(optimizer, **hps["lr_scheduler_kwargs"],)
else:
    scheduler = None

### Loss

In [12]:
criterion = {
    loss_name: str_loss_map[hps["loss"][loss_name]](
        hps["class_weights"][loss_name], 
        **hps["loss_kwargs"][loss_name]
    ) for loss_name in hps["loss"]
}

In [13]:
torch_class_weights = torch.FloatTensor(class_weights).cuda()
criterion['dx'] = torch.nn.CrossEntropyLoss(weight=torch_class_weights)
criterion['dx'].weight

tensor([1.0000, 1.2159, 1.2588, 1.3049, 1.3718, 1.8136], device='cuda:0')

### Metrics

In [14]:
tb_dir = os.path.join(experiment_fold_dir, "tensorboard")
tb_writer = SummaryWriter(tb_dir)
# tb_writer.add_graph(model, torch.zeros((1, 3) + hps["target_size"]).to(device))
history = defaultdict(list)

## Training

In [15]:
import warnings
warnings.filterwarnings("ignore")

iterate_through_dataloader_common_kwargs = {
    "model": model,
    "optimizer": optimizer,
    "criterion": criterion,
    "criterion_weight": hps["loss_factors"],
    "n_total_epochs": hps["num_epochs"],
    "metric_names": metric_names,
    "scheduler": scheduler,
    "device": device,
}

for epoch in range(1, hps["num_epochs"] + 1):
    start_time = time()
    running_metrics_train = iterate_through_attn_dataloader(
        dataloader_train,
        epoch=epoch,
        train_mode=True,
        **iterate_through_dataloader_common_kwargs,
    )
    running_metrics_test = iterate_through_attn_dataloader(
        dataloader_test,
        epoch=epoch,
        train_mode=False,
        force_enable_grad=hps["return_gradcam_attributes"],
        **iterate_through_dataloader_common_kwargs,
    )

    # Write and report history
    history["epoch"].append(epoch)
    for name in metric_names:
        history[f"{name}_train"].append(running_metrics_train[name])
        history[f"{name}_val"].append(running_metrics_test[name])
        tb_writer.add_scalar(f"{name}/train", running_metrics_train[name], epoch)
        tb_writer.add_scalar(f"{name}/test", running_metrics_test[name], epoch)
    if hps["lr_scheduler"] is not None:
        last_lr = scheduler.get_last_lr()[0]
        history["lr"].append(last_lr)
        tb_writer.add_scalar("lr", last_lr, epoch)
    save_json(experiment_fold_dir, "history.json", history)
    print_history_result(history, start_time)

    # Model checkpoint
    lastest_checkpoint_metric_value = history[f"{checkpoint_metric}_val"][-1]
    smaller_than_comparison = (
        lastest_checkpoint_metric_value < checkpoint_metric_value[checkpoint_metric]
    )
    larger_than_comparison = (
        lastest_checkpoint_metric_value > checkpoint_metric_value[checkpoint_metric]
    )
    if (
        larger_than_comparison
        if metric_larger_is_better[checkpoint_metric]
        else smaller_than_comparison
    ):
        torch.save(
            model.state_dict(),
            os.path.join(
                experiment_fold_dir,
                f"model_with_best_{checkpoint_metric}_at_epoch_{epoch}.pt",
            ),
        )
        print(
            "Model got better and was saved, with "
            f"{checkpoint_metric}_val = {lastest_checkpoint_metric_value:.3f}"
        )
        checkpoint_metric_value[checkpoint_metric] = lastest_checkpoint_metric_value
    if epoch == hps["num_epochs"]:
        torch.save(
            model.state_dict(),
            os.path.join(experiment_fold_dir, f"model_at_train_end.pt"),
        )

tb_writer.close()

Epoch 1/120: 100%|██████████| 16/16 [01:21<00:00,  5.09s/it]


Epoch 1: dx_train: 1.540 -- dx_val: 1.205 -- cx_train: 0.699 -- cx_val: 0.699 -- attn_train: 0.492 -- attn_val: 0.487 -- loss_train: 1.540 -- loss_val: 1.205 -- dx_acc_train: 0.390 -- dx_acc_val: 0.552 -- cx_f1_train: 0.288 -- cx_f1_val: 0.313 -- soft_iou_train: 0.107 -- soft_iou_val: 0.111 -- lr: 0.000 -- completed in 1.4 mins
Model got better and was saved, with loss_val = 1.205


Epoch 2/120: 100%|██████████| 16/16 [01:24<00:00,  5.26s/it]


Epoch 2: dx_train: 0.836 -- dx_val: 0.829 -- cx_train: 0.715 -- cx_val: 0.709 -- attn_train: 0.487 -- attn_val: 0.494 -- loss_train: 0.836 -- loss_val: 0.829 -- dx_acc_train: 0.716 -- dx_acc_val: 0.658 -- cx_f1_train: 0.312 -- cx_f1_val: 0.290 -- soft_iou_train: 0.114 -- soft_iou_val: 0.101 -- lr: 0.000 -- completed in 1.5 mins
Model got better and was saved, with loss_val = 0.829


Epoch 3/120: 100%|██████████| 16/16 [01:26<00:00,  5.38s/it]


...

Epoch 118: dx_train: 0.001 -- dx_val: 0.968 -- cx_train: 1.829 -- cx_val: 1.545 -- attn_train: 0.495 -- attn_val: 0.497 -- loss_train: 0.001 -- loss_val: 0.968 -- dx_acc_train: 1.000 -- dx_acc_val: 0.825 -- cx_f1_train: 0.314 -- cx_f1_val: 0.359 -- soft_iou_train: 0.136 -- soft_iou_val: 0.125 -- lr: 0.000 -- completed in 1.5 mins


Epoch 119/120: 100%|██████████| 16/16 [01:27<00:00,  5.45s/it]


Epoch 119: dx_train: 0.004 -- dx_val: 0.888 -- cx_train: 1.848 -- cx_val: 1.575 -- attn_train: 0.493 -- attn_val: 0.501 -- loss_train: 0.004 -- loss_val: 0.888 -- dx_acc_train: 0.998 -- dx_acc_val: 0.847 -- cx_f1_train: 0.311 -- cx_f1_val: 0.347 -- soft_iou_train: 0.137 -- soft_iou_val: 0.125 -- lr: 0.000 -- completed in 1.5 mins


Epoch 120/120: 100%|██████████| 16/16 [01:25<00:00,  5.31s/it]


Epoch 120: dx_train: 0.001 -- dx_val: 1.052 -- cx_train: 1.847 -- cx_val: 1.591 -- attn_train: 0.496 -- attn_val: 0.504 -- loss_train: 0.001 -- loss_val: 1.052 -- dx_acc_train: 1.000 -- dx_acc_val: 0.798 -- cx_f1_train: 0.316 -- cx_f1_val: 0.348 -- soft_iou_train: 0.127 -- soft_iou_val: 0.125 -- lr: 0.001 -- completed in 1.5 mins


## Save validation predictions

In [19]:
# model.load_state_dict(torch.load(f"{experiment_fold_dir}/model_at_train_end.pt"))
model.eval()

dict_keys = ['filename', 'predicted_dx', 'actual_dx']
dict_keys = np.concatenate((dict_keys, [f'actual_{cx}' for cx in characteristics]))
dict_keys = np.concatenate((dict_keys, [f'predicted_{cx}' for cx in characteristics]))
validation_dict = dict.fromkeys(dict_keys, np.array([]))

grad_cam_dir = f'{experiment_fold_dir}/gradcams'
if not os.path.exists(grad_cam_dir):
    os.mkdir(grad_cam_dir)

for batch in dataloader_test:
    images, dx_labels, cx_labels, mask_labels, filenames = batch
    dx_labels = [diagnoses[dx] for dx in dx_labels]
    
    predicted_dx, predicted_cx, predicted_attributes = model(images.to(device))
    predicted_dx = [diagnoses[dx] for dx in np.argmax(predicted_dx.detach().cpu().numpy(), axis=1)]
    predicted_cx = np.round(torch.sigmoid(predicted_cx).detach().cpu().numpy())
    predicted_attributes = predicted_attributes.detach().cpu().numpy()
    
    validation_dict['filename'] = np.concatenate((validation_dict['filename'], [Path(filename).name for filename in filenames]))
    validation_dict['predicted_dx'] = np.concatenate((validation_dict['predicted_dx'], predicted_dx))    
    validation_dict['actual_dx'] = np.concatenate((validation_dict['actual_dx'], dx_labels))
    
    for cx_idx in range(len(characteristics)):
        validation_dict[f'predicted_{characteristics[cx_idx]}'] = np.concatenate((validation_dict[f'predicted_{characteristics[cx_idx]}'], predicted_cx[:, cx_idx]))    
        validation_dict[f'actual_{characteristics[cx_idx]}'] = np.concatenate((validation_dict[f'actual_{characteristics[cx_idx]}'], cx_labels[:, cx_idx]))
        
        for image_idx in range(len(images)):
            mask_save_name = f'{grad_cam_dir}/{Path(filenames[image_idx]).name}_{characteristics[cx_idx]}.png'
            cv2.imwrite(mask_save_name, predicted_attributes[image_idx,cx_idx,::] * 255)

In [20]:
df = pd.DataFrame.from_dict(validation_dict).set_index('filename')
df.to_csv(f'{experiment_fold_dir}/final_model_predictions.csv')
df

,predicted_dx,actual_dx,actual_Closed comedo,actual_Dermatoglyph disruption,actual_Open comedo,actual_Papule,actual_Patch,actual_Plaque,actual_Pustule,actual_Scale,...,predicted_Closed comedo,predicted_Dermatoglyph disruption,predicted_Open comedo,predicted_Papule,predicted_Patch,predicted_Plaque,predicted_Pustule,predicted_Scale,predicted_Scar,predicted_Sun damage
filename,,,,,,,,,,,,,,,,,,,,,
viral-wart-15--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg,Viral warts,Viral warts,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
10556-P7070060.jpeg,Viral warts,Acne,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
043052HB.jpeg,Seborrheic dermatitis,Seborrheic dermatitis,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
halo5--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg,Vitiligo,Vitiligo,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
vitiligo-10--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg,Vitiligo,Vitiligo,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
facial-psoriasis06--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg,Acne,Psoriasis,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
496--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg,Actinic keratosis,Actinic keratosis,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
016139HB.jpeg,Acne,Actinic keratosis,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
044314HB.jpeg,Psoriasis,Psoriasis,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
